# Readme

1. Upload dataset file: Landcover Classification in Chesapeake 2020: https://portal.edirepository.org/nis/metadataviewer?packageid=knb-lter-vcr.379.1 
2. Transformations: Convert x and y coordinates to Laitude and Longitude
3. Subsetting data based nan values
4. Organize data based on classification (making upland forest file)
5. Subsetting data based on lat-lon box
6. Downloading data as csv

The final output is upland forest coordinates file (in specific region).

In [1]:
import xarray as xr
from pyproj import Transformer
from shapely.ops import transform
from shapely.geometry import Point
import numpy as np
from geopandas import GeoSeries
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
from pyproj import Transformer
from shapely.ops import transform
import csv 
import pandas as pd
import os

## 1. Upload dataset file

In [2]:
landcover = xr.open_dataset('/home/jovyan/sara_typrin/landcover/Landcover_year2020.tif')
#landcover.info()

Data is organized as a 3D array. Each x-y corresponds to a classification value between 0 and 6

In [3]:
#band = landcover['band_data']
#band

In [5]:
#data = landcover['band_data']
#data.max

## 2. Coordinate Transformation

Defining transformation that converts point to 'epsg:4326', the longitude latitue coordinate system.  

In [4]:
vcu = Point(-77.45, 37.55)
t = Transformer.from_crs(crs_from='epsg:32618', crs_to='epsg:4326', always_xy=True).transform
vcu_utm = transform(t, vcu)
print(vcu_utm.wkt)  # Prints easting, northing

POINT (-79.48943775842608 0.0003386772374478)


In [6]:
#vcu_utm

In [5]:
print(vcu_utm.wkt)  # Prints easting, northing

POINT (-79.48943775842608 0.0003386772374478)


Define landcover array variables: x_values, y_values, and data

In [84]:
x_values = landcover.x.values
y_values = landcover.y.values
data=landcover.band_data.values

#print(len(x_values))
#print(len(y_values))
print(data)

[[[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]]


Combine x and y dimensions into 1 dimension containing all classification values

In [7]:
flattened = data.stack(coords=("y", "x"))
#print(len(flattened))
#print(flattened[:5])

## 3. Taking Subset of Data

Most values in flattened dataset have no classification (nan). We are only interested in the classified values, so we take a subset of the not nan values. 

In [ ]:
print(flattened.values)

In [92]:
subset = flattened.dropna(dim="coords")
#print(subset.values[-5:])
subset[0]

<xarray.DataArray 'band_data' (coords: 11434272)> Size: 46MB
array([2., 1., 3., ..., 6., 6., 0.], dtype=float32)
Coordinates:
    band         int64 8B 1
    spatial_ref  int64 8B ...
  * coords       (coords) object 91MB MultiIndex
  * y            (coords) float64 91MB 4.4e+06 4.4e+06 ... 4.058e+06 4.058e+06
  * x            (coords) float64 91MB 4.537e+05 4.537e+05 ... 4.188e+05
Attributes:
    DataType:                Generic
    AREA_OR_POINT:           Area
    RepresentationType:      THEMATIC
    STATISTICS_COVARIANCES:  2.598902592411322
    STATISTICS_MAXIMUM:      6
    STATISTICS_MEAN:         2.881873459019
    STATISTICS_MINIMUM:      0
    STATISTICS_SKIPFACTORX:  1
    STATISTICS_SKIPFACTORY:  1
    STATISTICS_STDDEV:       1.6121112220971

In [12]:
# Get the MultiIndex from the 'coords' dimension
multiindex = subset.coords['coords'].values  # each element is (y, x)

# Unpack to get x and y values separately
y_values, x_values = zip(*multiindex)

# Convert to NumPy arrays (optional)
x_values = np.array(x_values)
y_values = np.array(y_values)


s = GeoSeries(map(Point, zip(x_values, y_values)))

In [28]:
print(len(s))

11434272


In [33]:
#Define a 5 point array of "s" to troubleshoot
s_short = s[-5:]
#print(s_short)

11434267    POINT (418680 4058370)
11434268    POINT (418740 4058370)
11434269    POINT (418770 4058370)
11434270    POINT (418770 4058340)
11434271    POINT (418800 4058340)
dtype: geometry


Create a list for the coordinates in the subset. Tranform the coordinate with the previously defined tranformation (t)

In [23]:
coords_utm = []
for point in s:
    transformed_point = transform(t, point)
    coords_utm.append(transformed_point)
#print(len(coords_utm))
#print(coords_utm)

11434272


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [36]:
print(len(coords_utm))
print(coords_utm[:5])

11434272
[<POINT (-75.54 39.745)>, <POINT (-75.54 39.745)>, <POINT (-75.539 39.745)>, <POINT (-75.539 39.745)>, <POINT (-75.538 39.745)>]


In [50]:
gdf = gpd.GeoDataFrame(geometry=coords_utm)

In [51]:
gdf[:5]

,geometry
0,POINT (-75.54017 39.74523)
1,POINT (-75.53982 39.74523)
2,POINT (-75.53947 39.74523)
3,POINT (-75.53912 39.74523)
4,POINT (-75.53842 39.74523)


In [52]:
gdf.geometry.x

0          -75.540173
1          -75.539822
2          -75.539472
3          -75.539122
4          -75.538422
              ...    
11434267   -75.909995
11434268   -75.909324
11434269   -75.908988
11434270   -75.908985
11434271   -75.908649
Length: 11434272, dtype: float64

In [26]:
all_subset_values = subset.values.flatten()
short_subset_values = subset_values[-5:]
#print(short_subset_values)

In [1]:
subset[0]

NameError: name 'subset' is not defined

In [125]:
lon = gdf.geometry.x
lat = gdf.geometry.y

coords_utm_da = xr.DataArray(
    data=subset[0],  # if subset is [[...]]
    dims=['points'],
    coords={
        'longitude': ('points', lon),
        'latitude': ('points', lat)
    }
)

### Select subset of the coordinates in Blackwater River Refuge.

In [128]:
coords_utm_subset = coords_utm_da.where(
    (coords_utm_da.longitude >= -76.35156992998947) &
    (coords_utm_da.longitude <= -75.68415049639572) &
    (coords_utm_da.latitude >= 38.19934796579719) &
    (coords_utm_da.latitude <= 38.67479854331683),
    drop=True
)

In [130]:
coords_utm_subset.values

array([2., 5., 5., ..., 2., 1., 3.], dtype=float32)

In [131]:
subset_values = coords_utm_subset.values

## 4. Organize Data Based on Classification

Values attatched to classification are as follows: water (0), farmland (1), urban area(2), upland forest (3), transition forest (4), marsh (5) and sandbar (6)

In [132]:
type_list = []

for i in range(len(subset_values)):
    if subset_values[i] == 0:
        type_list.append('water')
    elif subset_values[i] == 1:
        type_list.append('farmland')
    elif subset_values[i] == 2:
        type_list.append('urban area')
    elif subset_values[i] == 3:
        type_list.append('upland forest')
    elif subset_values[i] == 4:
        type_list.append('transition forest')
    elif subset_values[i] == 5:
        type_list.append('marsh')
    elif subset_values[i] == 6:
        type_list.append('sandbar')

#We should expect the same type list to contain all the same landcover types
type_list[-5:]

['farmland', 'farmland', 'urban area', 'farmland', 'upland forest']

In [140]:
len(coords_utm_subset.values)

1607424

In [144]:
len(coords_utm_subset)

1607424

In [151]:
lat = (coords_utm_subset.points.latitude)
lon =  (coords_utm_subset.points.longitude)

In [1]:
points = [Point(x, y) for x, y in zip(lon, lat)]
#points[:5]

NameError: name 'lon' is not defined

Create dataframe that has coordinates, type value, and type.

In [156]:
xyz_utm = pd.DataFrame({
    'point': points,  # list of (x, y) tuples
    'type_value': coords_utm_subset.values,
    'type': type_list
})

In [157]:
print(len(xyz_utm))
print(xyz_utm[-5:])

1607424
                                                point  type_value  \
1607419  POINT (-75.68581957282105 38.19947664347827)         1.0   
1607420   POINT (-75.6854769712009 38.19947864430668)         1.0   
1607421  POINT (-75.6851343695468 38.199480644135456)         2.0   
1607422  POINT (-75.68479176785877 38.19948264296461)         1.0   
1607423  POINT (-75.68444916613683 38.19948464079413)         3.0   

                  type  
1607419       farmland  
1607420       farmland  
1607421     urban area  
1607422       farmland  
1607423  upland forest  


Generate separate coordinate dataframes for each landcover type.

In [158]:
water_list = xyz_utm[xyz_utm['type'] == 'water']
farmland_list = xyz_utm[xyz_utm['type'] == 'farmland']
urban_list = xyz_utm[xyz_utm['type'] == 'urban area']
upland_forest_list = xyz_utm[xyz_utm['type'] == 'upland forest']
transition_forest_list = xyz_utm[xyz_utm['type'] == 'transition forest']
marsh_list = xyz_utm[xyz_utm['type'] == 'marsh']
sandbar_list = xyz_utm[xyz_utm['type'] == 'sandbar']

## 5. Downloading Files as csv

Code for downloading upland forest dataframe as csv:

In [162]:
upland_forest_list.to_csv('upland_forest_list_subset.csv', index=False)

In [159]:
len(upland_forest_list)

583995

Code for downloading upland forest datadrame as multiple files:

In [46]:
# Parameters
chunk_size = 5000
output_dir = "/home/jovyan/sara_typrin/landcover/data/upland_forest_list_5000"

# Create output folder if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Split the list into chunks
num_chunks = len(upland_forest_list) // chunk_size + 1
chunks = np.array_split(upland_forest_list, num_chunks)

# Save each chunk as a CSV
for i, chunk in enumerate(chunks):
    df = pd.DataFrame(chunk)  # Convert list chunk to DataFrame
    output_path = os.path.join(output_dir, f"output_chunk_{i}.csv")
    df.to_csv(output_path, index=False)

/srv/conda/envs/notebook/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Path for saving files:

In [49]:
directory_path =  "/home/jovyan/sara_typrin/landcover/data/upland_forest_list_5000"
file_count = 0
# Iterate through the contents of the directory
for item in os.listdir(directory_path):
    # Construct the full path to the item
    item_path = os.path.join(directory_path, item)
    # Check if the item is a file (and not a directory)
    if os.path.isfile(item_path):
        file_count += 1
print(f"Number of files in '{directory_path}': {file_count}")

Number of files in '/home/jovyan/sara_typrin/landcover/data/upland_forest_list_5000': 811
